In [1]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime

In [3]:
# Load the stock price dataset
stock_data= pd.read_csv("bse_data.csv")

print(stock_data.head())

         Date     Price      Open      High       Low    Vol. Change %
0  28-06-2024  79032.73  79457.58  79671.58  78905.89  17.24M   -0.27%
1  27-06-2024  79243.18  78758.67  79396.03  78467.34   9.60M    0.72%
2  26-06-2024  78674.25  78094.02  78759.40  77945.94   8.56M    0.80%
3  25-06-2024  78053.52  77529.19  78164.71  77459.60   8.83M    0.92%
4  24-06-2024  77341.08  76885.65  77423.02  76745.94  13.92M    0.17%


In [4]:
# Convert the date format from DD-MM-YYYY to YYYYMMDD (numerical type format)
def convert_date(date_str):
    for fmt in ["%d-%m-%Y", "%d/%m/%Y"]:
        try:
            return datetime.strptime(date_str, fmt).strftime("%Y%m%d")
        except ValueError:
            pass
    return None

stock_data['Date'] = stock_data['Date'].apply(convert_date)
print(stock_data.head())

       Date     Price      Open      High       Low    Vol. Change %
0  20240628  79032.73  79457.58  79671.58  78905.89  17.24M   -0.27%
1  20240627  79243.18  78758.67  79396.03  78467.34   9.60M    0.72%
2  20240626  78674.25  78094.02  78759.40  77945.94   8.56M    0.80%
3  20240625  78053.52  77529.19  78164.71  77459.60   8.83M    0.92%
4  20240624  77341.08  76885.65  77423.02  76745.94  13.92M    0.17%


In [5]:
# Load the news channel dataset
news_data = pd.read_csv("news_data.csv")
# print(news_data.head())

# Filtering the necessary data only
data_news = news_data[(news_data['publish_date'] >= 20190101)]
data_news = data_news.sort_values(by='publish_date', ascending=False)
data_news = data_news.reset_index(drop=True)

print(data_news)

        publish_date        headline_category  \
0           20230630                 city.goa   
1           20230630                    india   
2           20230630              city.mumbai   
3           20230630                    india   
4           20230630               city.delhi   
...              ...                      ...   
817239      20190101              city.mumbai   
817240      20190101          citizen.stories   
817241      20190101            city.varanasi   
817242      20190101             city.kolkata   
817243      20190101  life-style.fashion.buzz   

                                            headline_text  
0       Technicians to hold trial run of mini-EVs in P...  
1       Tamil Nadu women thanas are now kangaroo court...  
2                Teen & youth drown in well at Nalasopara  
3       Air fares drop; cheapest ticket from Delhi to ...  
4       From reel to real: Minor among 2 held for robbery  
...                                                

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\wwwdh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
sia = SentimentIntensityAnalyzer()
data_news['sentiment'] = data_news['headline_text'].apply(lambda x: sia.polarity_scores(x)['compound'])
print(data_news.head())

   publish_date headline_category  \
0      20230630          city.goa   
1      20230630             india   
2      20230630       city.mumbai   
3      20230630             india   
4      20230630        city.delhi   

                                       headline_text  sentiment  
0  Technicians to hold trial run of mini-EVs in P...     0.0000  
1  Tamil Nadu women thanas are now kangaroo court...     0.0000  
2           Teen & youth drown in well at Nalasopara    -0.3818  
3  Air fares drop; cheapest ticket from Delhi to ...    -0.2732  
4  From reel to real: Minor among 2 held for robbery     0.0000  


In [8]:
data_news = data_news.rename(columns={
    'publish_date': 'Date',
    'headline_category': 'Category',
    'headline_text': 'News',
    'sentiment': 'CompoundSentiment'
})

print(data_news.head())

       Date     Category                                               News  \
0  20230630     city.goa  Technicians to hold trial run of mini-EVs in P...   
1  20230630        india  Tamil Nadu women thanas are now kangaroo court...   
2  20230630  city.mumbai           Teen & youth drown in well at Nalasopara   
3  20230630        india  Air fares drop; cheapest ticket from Delhi to ...   
4  20230630   city.delhi  From reel to real: Minor among 2 held for robbery   

   CompoundSentiment  
0             0.0000  
1             0.0000  
2            -0.3818  
3            -0.2732  
4             0.0000  


In [9]:
def convert_date(date_obj):
    if isinstance(date_obj, str):
        for fmt in ["%d-%m-%Y", "%d/%m/%Y", "%Y-%m-%d"]:
            try:
                return datetime.strptime(date_obj, fmt).strftime("%Y%m%d")
            except ValueError:
                pass
    elif isinstance(date_obj, pd.Timestamp):
        return date_obj.strftime("%Y%m%d")
    return None

data_news['Date'] = data_news['Date'].apply(convert_date)
print(stock_data.dtypes)
print(data_news.dtypes)

Date         object
Price       float64
Open        float64
High        float64
Low         float64
Vol.         object
Change %     object
dtype: object
Date                  object
Category              object
News                  object
CompoundSentiment    float64
dtype: object


In [10]:
# merged_data = pd.merge(stock_data, data_news, on='Date')
stock_data['Sentiment'] = data_news['CompoundSentiment']
stock_data.dropna(inplace=True)
stock_data.drop(columns=['Date'], inplace=True)
print(stock_data.head())

      Price      Open      High       Low    Vol. Change %  Sentiment
0  79032.73  79457.58  79671.58  78905.89  17.24M   -0.27%     0.0000
1  79243.18  78758.67  79396.03  78467.34   9.60M    0.72%     0.0000
2  78674.25  78094.02  78759.40  77945.94   8.56M    0.80%    -0.3818
3  78053.52  77529.19  78164.71  77459.60   8.83M    0.92%    -0.2732
4  77341.08  76885.65  77423.02  76745.94  13.92M    0.17%     0.0000


In [11]:
print(stock_data.isnull().sum())

Price        0
Open         0
High         0
Low          0
Vol.         0
Change %     0
Sentiment    0
dtype: int64


In [12]:
print(stock_data.shape)
# merged_data = merged_data.fillna(method='ffill')

(1357, 7)


In [13]:
def convert_volume(volume_str):
    if 'M' in volume_str:
        return float(volume_str.replace('M', '')) * 1000000
    elif 'B' in volume_str:
        return float(volume_str.replace('B', '')) * 1000000000
    elif 'K' in volume_str:
        return float(volume_str.replace('K', '')) * 1000
    else:
        return float(volume_str)

# Apply the function to the 'Volume' column
stock_data['Vol.'] = stock_data['Vol.'].apply(convert_volume)
print(stock_data.head())

      Price      Open      High       Low        Vol. Change %  Sentiment
0  79032.73  79457.58  79671.58  78905.89  17240000.0   -0.27%     0.0000
1  79243.18  78758.67  79396.03  78467.34   9600000.0    0.72%     0.0000
2  78674.25  78094.02  78759.40  77945.94   8560000.0    0.80%    -0.3818
3  78053.52  77529.19  78164.71  77459.60   8830000.0    0.92%    -0.2732
4  77341.08  76885.65  77423.02  76745.94  13920000.0    0.17%     0.0000


In [14]:
def convert_change(change_str):
    if '%' in change_str:
        return float(change_str.replace('%', '')) / 100
    else:
        return float(change_str)

# Apply the function to the 'Volume' column
stock_data['Change %'] = stock_data['Change %'].apply(convert_change)
print(stock_data.head())

      Price      Open      High       Low        Vol.  Change %  Sentiment
0  79032.73  79457.58  79671.58  78905.89  17240000.0   -0.0027     0.0000
1  79243.18  78758.67  79396.03  78467.34   9600000.0    0.0072     0.0000
2  78674.25  78094.02  78759.40  77945.94   8560000.0    0.0080    -0.3818
3  78053.52  77529.19  78164.71  77459.60   8830000.0    0.0092    -0.2732
4  77341.08  76885.65  77423.02  76745.94  13920000.0    0.0017     0.0000


In [15]:
# Prepare data for Random Forest model
# merged_data['Price'] = merged_data['Price'].astype(str).str.replace(',', '').astype(float)
X = stock_data[['Open', 'High', 'Low', 'Vol.', 'Change %', 'Sentiment']]
y = stock_data['Price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on testing set
y_pred = rf_model.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Use the trained model to make predictions on new data
new_data = pd.DataFrame({'Open': [100], 'High': [120], 'Low': [90], 'Vol.': [1000], 'Change %': [2], 'sentiment': [0.5]})
# new_data = pd.get_dummies(new_data, columns=['sentiment'])
prediction = rf_model.predict(new_data)
print(f'Predicted stock price: {prediction[0]:.2f}')

Mean Squared Error: 58570.36
Predicted stock price: 27186.46


In [17]:
import numpy as np

In [19]:
# Calculate Direction Accuracy
direction_accuracy = np.mean((y_test - y_test.shift(1)) * (y_pred - y_test.shift(1)) > 0)
print(f'Direction Accuracy: {direction_accuracy:.2f}')

# Calculate Sharpe Ratio
y_test_series = pd.Series(y_test)
y_pred_series = pd.Series(y_pred)
returns = y_test_series.pct_change().dropna()
predicted_returns = y_pred_series.pct_change().dropna()
sharpe_ratio = (predicted_returns.mean() - returns.mean()) / predicted_returns.std()
print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

Direction Accuracy: 0.99
Sharpe Ratio: 0.00


In [32]:
# Calculate Direction Accuracy
def direction_accuracy(y_true, y_pred):
  correct_direction = 0
  for i in range(len(y_true)):
    if (y_true[i] > y_true[i-1] and y_pred[i] > y_pred[i-1]) or \
       (y_true[i] < y_true[i-1] and y_pred[i] < y_pred[i-1]):
      correct_direction += 1
  return (correct_direction / len(y_true)) * 100

# Calculate Daily Returns
daily_returns = y_test.pct_change().dropna()

# Calculate Risk-Free Rate (placeholder, replace with actual risk-free rate)
risk_free_rate = 0.02  # Adjust this value based on your assumption

# Calculate Average Daily Return
avg_daily_return = daily_returns.mean()

# Calculate Daily Volatility (standard deviation of daily returns)
daily_volatility = daily_returns.std()

# Calculate Sharpe Ratio
sharpe_ratio = (avg_daily_return - risk_free_rate) / daily_volatility

direction_accuracy_score = direction_accuracy(y_test.values, y_pred)

print(f'Direction Accuracy: {direction_accuracy_score:.2f}%')
print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

Direction Accuracy: 99.26%
Sharpe Ratio: 0.10


In [34]:
import numpy as np

# Function to calculate Direction Accuracy
def direction_accuracy(y_true, y_pred):
    # Determine the direction of actual and predicted changes
    actual_direction = np.sign(np.diff(y_true))
    predicted_direction = np.sign(np.diff(y_pred))
    
    # Calculate the percentage of matching directions
    accuracy = np.mean(actual_direction == predicted_direction)
    return accuracy

# Function to calculate Sharpe Ratio
def sharpe_ratio(y_true, y_pred, risk_free_rate=0.0):
    # Calculate daily returns from the true and predicted prices
    actual_returns = np.diff(y_true) / y_true[:-1]
    predicted_returns = np.diff(y_pred) / y_pred[:-1]
    
    # Calculate excess returns (subtracting risk-free rate)
    excess_returns = actual_returns - risk_free_rate
    
    # Calculate Sharpe Ratio (mean of excess returns divided by standard deviation)
    sharpe = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe

# Calculate Direction Accuracy
da = direction_accuracy(y_test.values, y_pred)
print(f'Direction Accuracy: {da:.2%}')

# Assuming a risk-free rate of 0% (adjust as needed)
rf_rate = 0.0

# Calculate Sharpe Ratio
sr = sharpe_ratio(y_test.values, y_pred, risk_free_rate=rf_rate)
print(f'Sharpe Ratio: {sr:.2f}')


Direction Accuracy: 99.26%
Sharpe Ratio: 0.16


In [ ]:
#############   Method 2 #############

In [20]:
data_stock = pd.read_csv("bse_data.csv")

def date_convert(date_str):
    for fmt in ["%d-%m-%Y", "%d/%m/%Y"]:
        try:
            return datetime.strptime(date_str, fmt).strftime("%Y%m%d")
        except ValueError:
            pass
    return None

def volume_convert(volume):
    if 'M' in volume:
        return float(volume.replace('M', '')) * 1000000
    elif 'B' in volume:
        return float(volume.replace('B', '')) * 1000000000
    elif 'K' in volume:
        return float(volume.replace('K', '')) * 1000
    else:
        return float(volume)

def change_convert(change):
    if '%' in change:
        return float(change.replace('%', '')) / 100
    else:
        return float(change)

data_stock['Date'] = data_stock['Date'].apply(date_convert)
data_stock['Vol.'] = data_stock['Vol.'].astype(str)
data_stock['Vol.'] = data_stock['Vol.'].apply(volume_convert)
data_stock['Change %'] = data_stock['Change %'].astype(str)
data_stock['Change %'] = data_stock['Change %'].apply(change_convert)
data_stock.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,20240628,79032.73,79457.58,79671.58,78905.89,17240000.0,-0.0027
1,20240627,79243.18,78758.67,79396.03,78467.34,9600000.0,0.0072
2,20240626,78674.25,78094.02,78759.40,77945.94,8560000.0,0.0080
3,20240625,78053.52,77529.19,78164.71,77459.60,8830000.0,0.0092
4,20240624,77341.08,76885.65,77423.02,76745.94,13920000.0,0.0017


In [21]:
channel_data = pd.read_csv("news_data.csv")

data_channel = channel_data[(news_data['publish_date'] >= 20190101)]
data_channel = data_channel.sort_values(by='publish_date', ascending=False)
data_channel = data_channel.reset_index(drop=True)
data_channel.head()

,publish_date,headline_category,headline_text
0,20230630,city.goa,Technicians to hold trial run of mini-EVs in P...
1,20230630,india,Tamil Nadu women thanas are now kangaroo court...
2,20230630,city.mumbai,Teen & youth drown in well at Nalasopara
3,20230630,india,Air fares drop; cheapest ticket from Delhi to ...
4,20230630,city.delhi,From reel to real: Minor among 2 held for robbery


In [22]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\wwwdh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [23]:
# data_channel.drop(0, inplace=True)
data_channel.drop('headline_category', axis = 1, inplace=True)
data_channel.head()

,publish_date,headline_text
0,20230630,Technicians to hold trial run of mini-EVs in P...
1,20230630,Tamil Nadu women thanas are now kangaroo court...
2,20230630,Teen & youth drown in well at Nalasopara
3,20230630,Air fares drop; cheapest ticket from Delhi to ...
4,20230630,From reel to real: Minor among 2 held for robbery


In [24]:
data_channel['headline_text'] = data_channel.groupby(['publish_date']).transform(lambda x : ' '.join(x)) 
data_channel = data_channel.drop_duplicates() 
data_channel.reset_index(inplace = True, drop = True)
data_channel.head()

,publish_date,headline_text
0,20230630,Technicians to hold trial run of mini-EVs in P...
1,20230629,Morning biz takes a hit amid downpour Economy ...
2,20230628,Come clean on graft charge: Congress to Vijaya...
3,20230627,HC asks Centre to find ways to bring sisters t...
4,20230626,airline travel co told to pay 50k for refund d...


In [25]:
%pip install textblob
from textblob import TextBlob

def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return  TextBlob(text).sentiment.polarity

In [26]:
data_channel['Subjectivity'] = data_channel['headline_text'].apply(getSubjectivity)
data_channel['Polarity'] = data_channel['headline_text'].apply(getPolarity)
data_channel.head()

,publish_date,headline_text,Subjectivity,Polarity
0,20230630,Technicians to hold trial run of mini-EVs in P...,0.343855,0.030601
1,20230629,Morning biz takes a hit amid downpour Economy ...,0.388498,0.038298
2,20230628,Come clean on graft charge: Congress to Vijaya...,0.364270,0.032572
3,20230627,HC asks Centre to find ways to bring sisters t...,0.323888,0.039497
4,20230626,airline travel co told to pay 50k for refund d...,0.380842,0.019580


In [27]:
SIA = SentimentIntensityAnalyzer()

data_channel['Compound'] = [SIA.polarity_scores(v)['compound'] for v in data_channel['headline_text']]
data_channel['Negative'] = [SIA.polarity_scores(v)['neg'] for v in data_channel['headline_text']]
data_channel['Neutral'] = [SIA.polarity_scores(v)['neu'] for v in data_channel['headline_text']]
data_channel['Positive'] = [SIA.polarity_scores(v)['pos'] for v in data_channel['headline_text']]
data_channel.head()

,publish_date,headline_text,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,20230630,Technicians to hold trial run of mini-EVs in P...,0.343855,0.030601,-0.9999,0.156,0.747,0.098
1,20230629,Morning biz takes a hit amid downpour Economy ...,0.388498,0.038298,-0.9999,0.152,0.765,0.083
2,20230628,Come clean on graft charge: Congress to Vijaya...,0.364270,0.032572,-0.9998,0.132,0.779,0.089
3,20230627,HC asks Centre to find ways to bring sisters t...,0.323888,0.039497,-1.0000,0.182,0.740,0.079
4,20230626,airline travel co told to pay 50k for refund d...,0.380842,0.019580,-0.9999,0.174,0.740,0.086


In [28]:
data_stock['Subjectivity'] = data_channel['Subjectivity']
data_stock['Polarity'] = data_channel['Polarity']
data_stock['Compound'] = data_channel['Compound']
data_stock['Negative'] = data_channel['Negative']
data_stock['Neutral'] = data_channel['Neutral']
data_stock['Positive'] = data_channel['Positive']

data_stock.dropna(inplace=True)
data_stock.drop(columns=['Date'], inplace=True)
data_stock.head()

,Price,Open,High,Low,Vol.,Change %,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,79032.73,79457.58,79671.58,78905.89,17240000.0,-0.0027,0.343855,0.030601,-0.9999,0.156,0.747,0.098
1,79243.18,78758.67,79396.03,78467.34,9600000.0,0.0072,0.388498,0.038298,-0.9999,0.152,0.765,0.083
2,78674.25,78094.02,78759.40,77945.94,8560000.0,0.0080,0.364270,0.032572,-0.9998,0.132,0.779,0.089
3,78053.52,77529.19,78164.71,77459.60,8830000.0,0.0092,0.323888,0.039497,-1.0000,0.182,0.740,0.079
4,77341.08,76885.65,77423.02,76745.94,13920000.0,0.0017,0.380842,0.019580,-0.9999,0.174,0.740,0.086


In [29]:
X1 = data_stock[['Open', 'High', 'Low', 'Vol.', 'Change %', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive']]
y1 = data_stock['Price']

# Split data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

# Create and train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X1_train, y1_train)

# Make predictions on testing set
y1_pred = rf_model.predict(X1_test)

# Evaluate model performance
mse1 = mean_squared_error(y1_test, y1_pred)
print(f'Mean Squared Error: {mse1:.2f}')

# Use the trained model to make predictions on new data
new_data_1 = pd.DataFrame({'Open': [100], 'High': [120], 'Low': [90], 'Vol.': [1000], 'Change %': [2], 'Subjectivity': [0.4], 'Polarity': [0.03], 'Compund': [0.06], 'Negative': [0.14], 'Neutral': [0.75], 'Positive': [0.06]})
# new_data = pd.get_dummies(new_data, columns=['sentiment'])
prediction1 = rf_model.predict(new_data_1)
print(f'Predicted stock price: {prediction1[0]:.2f}')

Mean Squared Error: 59534.93
Predicted stock price: 27610.87


In [31]:
# Calculate Direction Accuracy
direction_accuracy = np.mean((y1_test - y1_test.shift(1)) * (y1_pred - y1_test.shift(1)) > 0)
print(f'Direction Accuracy: {direction_accuracy:.2f}')

# Calculate Sharpe Ratio
y1_test_series = pd.Series(y1_test)
y1_pred_series = pd.Series(y1_pred)
returns = y1_test_series.pct_change().dropna()
predicted_returns = y1_pred_series.pct_change().dropna()
sharpe_ratio = (predicted_returns.mean() - returns.mean()) / predicted_returns.std()
print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

Direction Accuracy: 0.98
Sharpe Ratio: 0.00


In [42]:
# Calculate Direction Accuracy
def direction_accuracy(y1_true, y1_pred):
  correct_direction = 0
  for i in range(len(y1_true)):
    if (y1_true[i] > y1_true[i-1] and y1_pred[i] > y1_pred[i-1]) or \
       (y1_true[i] < y1_true[i-1] and y1_pred[i] < y1_pred[i-1]):
      correct_direction += 1
  return (correct_direction / len(y1_true)) * 100

# Calculate Daily Returns
daily_returns = y1_test.pct_change().dropna()

# Calculate Risk-Free Rate (placeholder, replace with actual risk-free rate)
risk_free_rate = 0.02  # Adjust this value based on your assumption

# Calculate Average Daily Return
avg_daily_return = daily_returns.mean()

# Calculate Daily Volatility (standard deviation of daily returns)
daily_volatility = daily_returns.std()

# Calculate Sharpe Ratio
sharpe_ratio = (avg_daily_return - risk_free_rate) / daily_volatility

direction_accuracy_score = direction_accuracy(y1_test.values, y1_pred)

print(f'Direction Accuracy: {direction_accuracy_score:.2f}%')
print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

Direction Accuracy: 98.90%
Sharpe Ratio: 0.16


In [40]:
import numpy as np

# Function to calculate Direction Accuracy
def direction_accuracy(y1_true, y1_pred):
    # Determine the direction of actual and predicted changes
    actual_direction = np.sign(np.diff(y1_true))
    predicted_direction = np.sign(np.diff(y1_pred))
    
    # Calculate the percentage of matching directions
    accuracy = np.mean(actual_direction == predicted_direction)
    return accuracy

# Function to calculate Sharpe Ratio
def sharpe_ratio(y1_true, y1_pred, risk_free_rate=0.0):
    # Calculate daily returns from the true and predicted prices
    actual_returns = np.diff(y1_true) / y1_true[:-1]
    predicted_returns = np.diff(y1_pred) / y1_pred[:-1]
    
    # Calculate excess returns (subtracting risk-free rate)
    excess_returns = actual_returns - risk_free_rate
    
    # Calculate Sharpe Ratio (mean of excess returns divided by standard deviation)
    sharpe = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe

# Calculate Direction Accuracy
da = direction_accuracy(y1_test.values, y1_pred)
print(f'Direction Accuracy: {da:.2%}')

# Assuming a risk-free rate of 0% (adjust as needed)
rf_rate = 0.0

# Calculate Sharpe Ratio
sr = sharpe_ratio(y1_test.values, y1_pred, risk_free_rate=rf_rate)
print(f'Sharpe Ratio: {sr:.2f}')

Direction Accuracy: 98.89%
Sharpe Ratio: 0.16
